In [2]:
import anthropic
import pandas as pd

client = anthropic.Anthropic()

In [14]:
def get_answer_prompt(problem:str) -> str:
    prompt = """Write Python code to solve this problem:

    {problem}
    
    Don't forget the imports.""".format(problem=problem)
    return prompt

In [15]:
from datasets import load_dataset

h_eval = load_dataset('openai/openai_humaneval')["test"]

In [16]:
problem = h_eval[0]["prompt"]
test = h_eval[0]["test"]

In [17]:
print(test)



METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False




In [18]:
import re

def extract_python_code(text):
    pattern = r"```python([\s\S]*?)```"
    matches = re.findall(pattern, text)
    return matches[0].strip()

In [19]:
messages = [{"role":"user", "content":[{"type":"text", "text":get_answer_prompt(problem)}]}]
for i in range(5):
    answer = client.messages.create(model="claude-3-5-sonnet-20241022",
        max_tokens=1000, temperature=0, messages=messages)
    messages[0]["content"].append({"type":"text", "text":extract_python_code(answer.content[0].text)})
    messages[0]["content"].append({"type":"text", "text":"write more optimized code"})


In [ ]:
import contextlib
import io
import time
from typing import Dict

class TimeoutException(Exception):
    pass


class WriteOnlyStringIO(io.StringIO):
    """StringIO that throws an exception when it's read from"""

    def read(self, *args, **kwargs):
        raise IOError

    def readline(self, *args, **kwargs):
        raise IOError

    def readlines(self, *args, **kwargs):
        raise IOError

    def readable(self, *args, **kwargs):
        """Returns True if the IO object can be read."""
        return False


class redirect_stdin(contextlib._RedirectStream):  # type: ignore
    _stream = "stdin"


@contextlib.contextmanager
def swallow_io():
    stream = WriteOnlyStringIO()
    with contextlib.redirect_stdout(stream):
        with contextlib.redirect_stderr(stream):
            with redirect_stdin(stream):
                yield

def unsafe_execute(problem: Dict, completion: str):
    check_program = (
        completion
        + "\n"
        + problem["test"]
        + "\n"
        + f"check({problem['entry_point']})"
    )
    try:
        exec_globals = {}
        with swallow_io():
            start = time.time()
            exec(check_program, exec_globals)
            time_taken = time.time() - start
        return ("passed", time_taken)
    except BaseException as e:
        return ("failed: {e}", 0)

In [45]:
problem = h_eval[0]
completion = extract_python_code(answer.content[0].text)
unsafe_execute(problem, completion)

('passed', 0.0)